# **Machine Learning Applications Final Project Dash**

Date: May 9, 2023

Authors: Óscar Montoya Amores (100451858), Javier Serrano Sánchez-Ocaña (100451673), Miguel Ángel Ponce Martínez (100451309)

#### **LIBRARIES IMPORT**

In [1]:
import joblib

from jupyter_dash import JupyterDash
from dash import dash, dcc, html, Input, Output

import plotly.express as px
import pandas as pd
import numpy as np
# import gensim
import pyLDAvis.gensim as gensimvis
import pyLDAvis

from io import BytesIO

import pandas as pd
from wordcloud import WordCloud
import base64

PLOTS A IMPLEMENTAR, BORRAR CUANDO ESTE ACABADO:

-LDA CON LOS TÓPICOS DEPENDIENDO DE CADA GENERO, ES DECIR PODER ELEGIR QUE GENERO PLOTEAR Y QUE SALGAN LAS PALABRAS Q CONFORMAN EL TOPICO

In [2]:
df = pd.read_csv('clean_songs.csv')
# print(df.columns)

ldag = joblib.load("lda_model.joblib")

from gensim.corpora import Dictionary

dictionary_file = "dictionary.gensim"
D = Dictionary.load(dictionary_file)
from gensim import corpora
# load the corpus from file
corpus = corpora.MmCorpus('corpus.mm')

corpus_words = [[D[word_id] for word_id, count in doc] for doc in corpus]

# print(type(ldag))
# print(type(corpus))
# print(corpus_words[0])
# print(type(D))
# print("wa" in corpus_words[0])

# Visualize topics using pyLDAvis
vocab = {D[i]: i for i in range(len(D))}
vocab2 = {i: D[i] for i in range(len(D))}
term_frequency = {D[i]: freq for i, freq in D.cfs.items()}
term_frequency = pd.Series(term_frequency)
vis = gensimvis.prepare(ldag, corpus, D, mds='tsne', vocab=vocab2, sort_topics=False, term_frequency=term_frequency)

In [3]:
#0. Obtain genres
genres = df['type'].unique().tolist()
print(genres)
#1. Obtain the Indices of each song genre
typeSer = df['type']
rock_idx  = typeSer[typeSer == 'rock'].index
print(rock_idx)


['rock', 'country', 'R&B', 'rap & hip hop']
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       953, 954, 955, 956, 957, 958, 959, 960, 961, 962],
      dtype='int64', length=963)


In [4]:
dfm = pd.DataFrame({'word': ['apple', 'pear', 'orange','Nano'], 'freq': [1,3,9,20]})
dfm2 = pd.DataFrame({'word': ['apple', 'pear', 'orange','Vettel'], 'freq': [1,3,9,20]})

In [5]:
# Initialize the Dash app
app = JupyterDash(__name__)

server = app.server

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("Song Lyrics Wordcloud by Genre"),
    dcc.Dropdown(
        id='dropdown_words',
        options=[{'label': f'Genre: {i}', 'value': i} for i in genres],
        value=genres[0]
    ),
    html.Img(id="image_wc"),
    html.H1("Song Genre Topic Modeling"),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': 'Topic {}'.format(i), 'value': i} for i in range(ldag.num_topics)],
        value=0
    ),
    dcc.Graph(id='topic-type-plot'),
    html.Iframe(
        id='pyldavis',
        srcDoc=pyLDAvis.prepared_data_to_html(vis),
        style={'width': '100%', 'height': '500px'}
    )
])

#Creation of WordCloud
@app.callback(
    Output('image_wc', 'src'), 
    [Input('image_wc', 'id'), 
     Input('dropdown_words', 'value')]
)

def make_image(b, gr):
    img = BytesIO()
    if gr == 'rock':
        plot_wordcloud(data=dfm).save(img, format='PNG')
    else:
        plot_wordcloud(data=dfm2).save(img, format='PNG')
    return 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())

def plot_wordcloud(data):
    d = {a: x for a, x in data.values}
    wc = WordCloud(colormap='prism', background_color='black', width=1900, height=640)#.generate(data)
    wc.fit_words(d)
    return wc.to_image()

# Define the update functions for the Dash app
@app.callback(
    Output('pyldavis', 'srcDoc'),
    Input('dropdown', 'value')
)
def update_pyldavis(selected_topic):
    return pyLDAvis.prepared_data_to_html(vis)

@app.callback(
    Output('topic-type-plot', 'figure'),
    Input('dropdown', 'value')
)
def update_topic_type_plot(selected_topic):
    # Get the top 20 words for the selected topic
    top_words = ldag.show_topic(selected_topic, topn=2)
    # print(top_words)
    top_words = [word[0] for word in top_words]
    if "wa" in top_words:
        top_words.remove("wa")
    # print(top_words)
    # Find the indices of the songs in the corpus that contain any of the top 20 words for the selected topic
    indices = []
    # print(len(corpus_words))
    """
    i = 0
    while i<len(corpus_words):
        for word in top_words:
            if word!="wa":
                if word in corpus_words[i]:
                    #print(word)
                    indices.append(i)
                    break
        i+=1
    """

    for i in range(len(corpus_words)):
        j=0
        for word in top_words:
            if word in corpus_words[i]:
                j+=1
        if j==len(top_words):
            indices.append(i)
            
    # print(indices)
    # Filter the dataframe to include only songs with indices in the list of indices
    topic_df = df[df.index.isin(indices)]
    # print(topic_df)
    # Check if any songs match the selected topic
    if topic_df.empty:
        return {'data': [], 'layout': {}}
    # Get the genre of each song in the selected topic
    genres = [genre for genre in topic_df['type']]
    # Count the number of songs in each genre for the selected topic
    genre_counts = pd.Series(genres).value_counts().sort_index()
    # Create the plot
    fig = {
        'data': [{'x': genre_counts.index, 'y': genre_counts.values, 'type': 'bar', 'marker': {'color': ['red', 'green', 'blue', 'orange']}}],
        'layout': {'title': f'Topic {selected_topic} {top_words} Song Genres by LDA' ,'xaxis': {'title': 'Genre'}, 'yaxis': {'title': 'Count'}}
    }
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



Dash is running on http://127.0.0.1:8050/



c:\Users\migue\anaconda4\envs\finalML\lib\site-packages\jupyter_dash\jupyter_app.py:310: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread.setDaemon(True)
